# SAN Assignment - Generalized linear models

Author : Your Name \
Email  : you@fel.cvut.cz

#### Introduction
The aim of this assignment is to practice constructing linear models. You will start with a simple linear model. You will evaluate and interpret it (1p). Consequently, your task will be to improve this model using generalized linear models (GLMs) and feature transformations. You will get 1p for proposal and evaluation of GLM (family, evaluation, interpretation), 1p for correct feature transformations, 1p for proposal and justification of the final model and eventually, 1p for comprehensive evaluation of all the model improvements (ablation study through cross-validation, note that the previous evaluations must be done without cross-validation).

#### Input data 
In this assignment, you will work with a student dataset. The dataset contains 200 samples and 4 features: *num_awards* is the outcome variable and indicates the number of awards earned by students in a year, *math* is a continuous predictor variable and represents students’ scores on their math final exam, *prog* is a categorical predictor variable with three levels indicating the type of program in which the students were enrolled (1 = “General”, 2 = “Academic” and 3 = “Vocational”), and *work* is a continuous predictor that gives the number of hours that students spent at work on average per week.

#### Load the necessary libraries and the dataset

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats
from collections import defaultdict
import scipy.stats
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold

df = pd.read_csv("study_data.csv")
X = df.loc[:, df.columns != "num_awards"]
X = sm.add_constant(X)
y = df.num_awards.to_numpy()

#### Simple linear model
Let us start with an ordinary linear model with no feature transformations. Explain how far the model works (does it meet formal assumptions?, does it overcome the null model?). Which predictors would you keep there and which of them are not useful? Use the standard evaluation procedures that we have for linear models (notice, that GML with Gaussian family corresponds to an OLS model, only the summary is slightly more general).


In [ ]:
res = sm.OLS(endog=y, exog=X).fit()
print(res.summary())
res = sm.GLM(endog=y, exog=X, family=sm.families.Gaussian()).fit()
print(res.summary())

plt.scatter(y, res.predict(X))
plt.ylabel("predicted no. awards")
plt.xlabel("real no. awards")
plt.ylim((-1, 6))
plt.show()


try:
    sm.graphics.plot_fit(res, "math")
except AttributeError:
    pass
finally:
    plt.show()

**Add your verbal summary here (1p)**:

### Generalized linear model
Now, the goal is to implement a generalized linear model that conceptually fits the given task. Do not transform the predictors yet, use them as they are, or omit them from the model. Once you obtain your model, interpret the effect of the *math* predictor on the outcome. How (according to your model) increasing a math score by one point affects the number of awards won? 

Explain why the model overcomes the previous linear model, or justify that the generalized model is not needed. Compare the models theoretically as well as technically in terms of a proper quality measure(s). Note: The difference between the models generally cannot be statistically tested.

Because in Python you cannot use *anova* function refer for example to the [[Likelihood ration test (LTR)|https://www.statology.org/likelihood-ratio-test-in-python/]] when comparing **nested** models, which you can easily implement yourself.

In [ ]:
#### add your code here ####


**Add your verbal summary here (1p)**:

### Feature transformations and final model
*prog* and *work* did not prove to be effective predictors so far. Visualize these predictors as well as their relationship with the outcome variable. Based on the observations, propose suitable transformations for them (or, justify that they are truly uninformative for prediction of *num_awards*) and implement them into the best model found by now. Use the *compareGLM()* function to validate that your new GLMs indeed improved over the simple one.

To achieve the same results as with R formulas (e.g. polynomial regression like *num_awards ~ poly(...)*) in Python you need to transform your features explicitly before passing them to your model. You do this by modifying the exog data in X.
For tools to do polynomial or spline transformations refer for example to sklearn:
[[SplineTransformer|https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.SplineTransformer.html]]
[[PolynomialFeatures|https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html]]




In [ ]:
cols = ["num_awards", "prog"]
df_transformed = df[cols].groupby(cols).size().reset_index(level=-1)
programs = pd.unique(df_transformed.prog).tolist()
pd.concat([
    df_transformed.loc[df_transformed.prog == p, 0]
    for p in programs
], axis=1).set_axis(programs, axis="columns").plot.bar()
plt.title("No. of students per no. of awards in each program")
plt.show()

for p in programs:
    plt.scatter(df.work.loc[df.prog == p], df.num_awards.loc[df.prog == p], label=p)
plt.title("Number of awards by amount of work and program")
plt.xlabel("work")
plt.ylabel("awards")
plt.legend()
plt.show()

#### add your code here ####


**Add your verbal summary here (2p)** :


### Ablation study through cross-validation
Recap all the models considered previously and evaluate them through cross-validation. You can start with the most simple null model and gradually add the previously discussed improvements. See their role in terms of MAE, RMSE and other commonly used criteria. The procedure outlined below proposes to work with the preprepared *train_with_cv* function, you can only add more models to evaluate and compare.

In [ ]:
def test_glm_with_cv(X, y, model_type, model_name, model_family=sm.families.Gaussian(), n_folds=10):
    """
    You can use this function to test your GLM models in CV settings. If you want, you can edit
    this function, e.g. to have more comparison metrics, but justify it well in your commentary.

    :param X: your independent variables / features
    :param y: your response variable
    :param model_name: just for convenience you can name your models
    :param model_family: family of your GLM model (from statsmodels.families),
                        the default corresponds to an OLS model
    :param n_folds: number of folds to perform in your CV
    :return: returns a DataFrame with all the metrics for a particular model
    """
    assert model_type in ["lm", "glm"]
    metrics = {
        "mse": mean_squared_error,
        "mae": mean_absolute_error,
        "r2": r2_score,
    }

    skf = KFold(n_splits=n_folds, shuffle=True, random_state=0)
    metrics_results = defaultdict(list)
    for train_idxs, test_idxs in skf.split(X, y):
        model = sm.GLM(endog=y[train_idxs], exog=X[train_idxs], family=model_family)
        fit = model.fit()
        cmp = y[test_idxs], fit.predict(X[test_idxs])
        for name, m in metrics.items():
            metrics_results[name].append(m(*cmp))

    means = {f"{n}_mean": [pd.Series(m).mean()] for n, m in metrics_results.items()}
    sds = {f"{n}_sd": [pd.Series(m).std()] for n, m in metrics_results.items()}

    res = pd.DataFrame({**means, **sds}).set_axis([model_name], axis="rows")
    return res

In [ ]:
# Here you have an example of the null lm model trained and tested with CV

# the sm.add_constant prepared a column "const" of ones for the intercept estimation
test_glm_with_cv(X = X.const.to_numpy(),
                  y = df.num_awards.to_numpy(),
                  model_type = "lm",
                  model_name="mean_model")

#### add your code here ####


**Add your verbal summary here (1p)**:
